In [1]:
import numpy as np
import dask.array as da
from pymks.fmks.correlations import auto_correlation
from pymks.fmks.pair_correlations import dist_from_center, paircorr_from_twopoint

## Check that the dist_from_center function works

The maximum value of a two-point correlation will occur at the center. Therefore, if I find the max along the sample axis of the two point correlations and I compare that to the value where the distance is 0- then those two values should match.

The below test is for two samples of a 3-dimensional structure.

In [2]:
C = da.random.randint(0,2, [2,3,3,3])

# 2 samples, 3x4 grids
C.shape

(2, 3, 3, 3)

In [3]:
D = dist_from_center(C, axes=[1,2,3])
D.shape

(2, 3, 3, 3)

In [4]:
tps = auto_correlation(C).compute()

print(tps.shape, D.shape)

assert len(np.argwhere(D==0)) == D.shape[0]
assert all(tps[D==0] == np.amax(tps, axis=(1,2,3)))

(2, 3, 3, 3) (2, 3, 3, 3)


## Run a pair correlation check for 2d

In this check I verify that the base pair correlation function works. I am also checking that it is possible to set a threshold on the radius and interpolate to regularly spaced radius values. The accuracy in the 2d case should extend to any number of dimensions since the function flattens the arrays before computing the pair correlations.

In [5]:
C = np.array([
    [
        [0.2, 0.4, 0.3],
        [0.4, 0.5, 0.5],
        [0.2, 0.5, 0.3]
    ],
    [
        [0.1, 0.2, 0.3],
        [0.2, 0.6, 0.4],
        [0.1, 0.4, 0.3]
    ]
])

pc_correct = np.array([
    [0, 0.5, 0.6],
    [1, 0.45, 0.3],
    [np.sqrt(2), 0.25, 0.2]
])

pc_correct_cut = np.array([
    [0, 0.5, 0.6],
    [1, 0.45, 0.3]
])

pc_correct_interped = np.array([
    [0, 0.5, 0.6],
    [np.sqrt(2), 0.25, 0.2]
])

print(C.shape, pc_correct.shape)

(2, 3, 3) (3, 3)


In [6]:
assert np.allclose(paircorr_from_twopoint(C), pc_correct)
assert np.allclose(paircorr_from_twopoint(C, cutoff_r=1.01), pc_correct_cut)
assert np.allclose(paircorr_from_twopoint(C, interpolate_n=2), pc_correct_interped)

## Run pair correlations for many 3d samples

In the following check, I verify that the `paircorr_from_twopoint` function works for 3d microstructures, dask inputs, and numpy inputs.

### Dask Workflow

In [7]:
# 100 5x5x5 microstructures
C = da.random.random((100,5,5,5))

assert paircorr_from_twopoint(C) is not None

### NumPy Workflow

In [8]:
# 100 5x5x5 microstructures
C = np.random.rand(100,5,5,5)

assert paircorr_from_twopoint(C) is not None